In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.01
seed = 0
alpha = 1e-3
lamda = 1e-3
eta = 1e-2
mu = 1e-2
no_users = sum(cluster_sizes)
batch_size = 50
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.11111111 0.         0.         0.14285714 0.
  0.         0.14285714 0.14285714 0.         0.         0.
  0.         0.         0.         0.         0.         0.11111111
  0.         0.14285714]
 [0.11111111 0.         0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.11111111 0.         0.125      0.125      0.
  0.125      0.         0.125      0.         0.         0.
  0.         0.         0.         0.         0.125      0.
  0.         0.125     ]
 [0.         0.11111111 0.125      0.         0.         0.16666667
  0.125      0.         0.14285714 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.14285714 0.11111111 0.125      0.         0.         0.
  0.125      0.16666667 0.         0.         0.         0.
  0.         0.         0.         0

In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
degree_list = [G.degree(i) for i in range(no_users)]
print(degree_list)

for i in G.neighbors(0):
    print(i)

[5, 6, 8, 7, 5, 5, 4, 7, 3, 6, 1, 5, 3, 4, 5, 3, 6, 6, 8, 3]
1
3
6
7
9


In [6]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.size)
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split





X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')


X, y = random_split(X_train, y_train, no_users, 1234)

In [7]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0, 3.0, 3.0])
W2 = np.array([-2.0, 2.0, 3.0, -3.0])
W3 = 2 * W1
W4 = 2  * W2
W = [W1, W2]
m = 200
n = 4

scaler = [1.0, -1.0]

noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i ]) + np.random.normal(0,noise_sd)
        data = X[count]
        data[:, 0:4] *= scaler[i]
        datapoints[count] = {
                'features': data,
                'degree': node_degree(count, G),
                'label': y[count],
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [8]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [9]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(9, 4, bias=False)
        self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        #output = self.fc3(x)
        return output

In [10]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [11]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        #new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        #vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(48.5478, grad_fn=<MseLossBackward0>) tensor([-0.3470,  0.2523,  1.7713, -0.8372,  0.8539,  0.5348,  0.7210, -0.8715,
         0.0837,  0.7392, -0.5987,  0.5631, -0.9992,  0.8662,  0.7514,  0.9207,
         0.2095,  1.8827, -2.2354,  1.8028,  0.5331,  1.0187, -0.5971, -0.3320,
        -0.7662, -0.6368, -3.5024, -0.2368,  0.7241, -0.6128,  0.1144, -0.8239,
        -0.7873, -0.6590,  2.3783,  0.7408, -1.0362, -1.3526, -1.9344, -3.2982]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(53.2632, grad_fn=<MseLossBackwar

torch.Size([50, 1])
torch.Size([50, 1])
3 tensor(44.3275, grad_fn=<MseLossBackward0>) tensor([-1.4582,  1.1119,  2.0143, -1.0773,  0.6216,  0.5742,  0.6529,  0.8887,
        -0.5158,  0.4361, -0.1033,  0.5256, -1.1289,  1.1783,  1.1414,  1.2277,
         0.3584,  1.5712, -2.1064,  1.6151,  1.0772,  0.2488, -0.2068, -0.0297,
        -0.2447, -0.1095, -2.6379, -0.2745,  0.7910,  0.1867, -0.1392, -0.0426,
        -0.3023, -0.0393,  1.2926,  0.8497, -1.1144, -1.2262, -1.6318, -1.9447]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
3 tensor(72.6023, grad_fn=<MseLossBackwar

        -0.7221, -0.0857,  1.0538,  0.7574, -1.3895, -1.1931, -1.9930, -2.1967]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
6 tensor(54.2208, grad_fn=<MseLossBackward0>) tensor([-0.4398,  0.2416,  1.6615, -0.5253,  0.5632,  0.2384,  0.3826,  0.4706,
        -0.3337,  0.3259, -0.1812,  0.5993, -1.7076,  1.0305,  0.7566,  1.1189,
         1.7011,  1.0618, -2.3580,  2.0360,  0.9187, -0.1013,  0.7036,  1.1450,
         0.5807, -1.4447, -2.9616, -0.0066,  0.3231,  0.2716, -0.2311, -0.6477,
        -0.7541, -0.5339,  2.6250,  0.7899, -0.8014, -1.2834, -1.9725, -3.3706]) 

torch.Size([50, 1])
torch.Size([50, 1])
9 tensor(56.7572, grad_fn=<MseLossBackward0>) tensor([-0.5785,  0.2656,  3.3907, -0.2683,  0.1023,  0.1971, -0.1172,  0.2133,
        -0.0290,  0.8322, -0.5512,  0.3451, -1.1951,  0.8637,  0.8191,  1.0500,
         1.1310,  0.7502, -2.8864,  2.0864,  2.0721,  0.7380, -0.7417,  0.1730,
        -1.0250, -0.9708, -2.5493, -0.4815,  0.9874, -0.7138, -0.5699, -0.4095,
        -0.4800, -0.1218,  2.4522,  0.1586, -1.5514, -0.9161, -2.2711, -3.5592]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
9 tensor(54.9587, grad_fn=<MseLossBackwar

        -0.2735,  0.1847,  1.3872, -0.3643, -1.5396, -1.3106, -1.2146, -2.2533]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
12 tensor(45.6119, grad_fn=<MseLossBackward0>) tensor([-0.3424,  0.4211,  1.1742, -0.8456,  0.7764,  0.6515,  0.7106,  0.3070,
         0.4539,  0.3019, -0.2493,  0.6548, -1.8096,  1.6734,  1.1914,  1.7863,
         0.0902,  0.9624, -2.7431,  2.2244, -0.4050,  0.4287, -0.4894,  0.1019,
        -0.6103, -0.5838, -1.4356, -0.3633,  0.6598,  0.5126,  0.4282, -0.9356,
        -0.6974, -0.7994,  1.6460, -0.3275, -0.7987, -1.3402, -1.4499, -2.0401])

...

97 tensor(48.5478, grad_fn=<MseLossBackward0>) tensor([-0.3470,  0.2523,  1.7713, -0.8372,  0.8539,  0.5348,  0.7210, -0.8715,
         0.0837,  0.7392, -0.5987,  0.5631, -0.9992,  0.8662,  0.7514,  0.9207,
         0.2095,  1.8827, -2.2354,  1.8028,  0.5331,  1.0187, -0.5971, -0.3320,
        -0.7662, -0.6368, -3.5024, -0.2368,  0.7241, -0.6128,  0.1144, -0.8239,
        -0.7873, -0.6590,  2.3783,  0.7408, -1.0362, -1.3526, -1.9344, -3.2982]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
97 tensor(53.2632, grad_fn=<MseLossBackward0>) tensor([-1.1753,  0.5564,  3.6616

97 tensor(54.2208, grad_fn=<MseLossBackward0>) tensor([-0.4398,  0.2416,  1.6615, -0.5253,  0.5632,  0.2384,  0.3826,  0.4706,
        -0.3337,  0.3259, -0.1812,  0.5993, -1.7076,  1.0305,  0.7566,  1.1189,
         1.7011,  1.0618, -2.3580,  2.0360,  0.9187, -0.1013,  0.7036,  1.1450,
         0.5807, -1.4447, -2.9616, -0.0066,  0.3231,  0.2716, -0.2311, -0.6477,
        -0.7541, -0.5339,  2.6250,  0.7899, -0.8014, -1.2834, -1.9725, -3.3706]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
97 tensor(56.7572, grad_fn=<MseLossBackward0>) tensor([-0.5785,  0.2656,  3.3907

98 tensor(46.8444, grad_fn=<MseLossBackward0>) tensor([-0.6847,  0.4322,  2.9137, -0.3601, -0.4910,  0.0381, -0.6553,  0.5024,
        -0.6456,  0.7998, -0.6238,  0.7683, -2.0511,  1.5168,  1.2284,  1.6623,
         1.2386,  1.1638, -2.8854,  2.3294,  0.9668,  1.4833, -1.1850, -0.4777,
        -1.4092, -0.9460, -2.8588,  0.4911, -0.2834, -0.0360, -1.3643,  0.4974,
         0.5757,  0.7148,  1.8643,  0.9043, -1.5223, -1.5837, -2.1258, -2.1235]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
98 tensor(44.3275, grad_fn=<MseLossBackward0>) tensor([-1.4582,  1.1119,  2.0143

98 tensor(45.6119, grad_fn=<MseLossBackward0>) tensor([-0.3424,  0.4211,  1.1742, -0.8456,  0.7764,  0.6515,  0.7106,  0.3070,
         0.4539,  0.3019, -0.2493,  0.6548, -1.8096,  1.6734,  1.1914,  1.7863,
         0.0902,  0.9624, -2.7431,  2.2244, -0.4050,  0.4287, -0.4894,  0.1019,
        -0.6103, -0.5838, -1.4356, -0.3633,  0.6598,  0.5126,  0.4282, -0.9356,
        -0.6974, -0.7994,  1.6460, -0.3275, -0.7987, -1.3402, -1.4499, -2.0401]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([4, 1])
torch.Size([4, 1])
98 tensor(39.3987, grad_fn=<MseLossBackward0>) tensor([-1.7608,  1.6497, -0.3491, 

        -1.3809, -0.9530,  2.2351,  2.0084, -0.9422, -1.2562, -2.1217, -3.3123]) tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
99 tensor(47.5602, grad_fn=<MseLossBackward0>) tensor([-0.2632, -0.0716,  2.3448, -0.7451,  0.0638, -0.2418, -0.0732,  0.7007,
        -0.2045,  1.0157, -0.7044,  0.2885, -0.9572,  0.8589,  0.4627,  0.8170,
         0.5912,  2.0071, -2.3329,  1.6991,  0.8287,  0.6592, -0.5660,  0.0061,
        -0.7934, -0.7673, -3.1640, -0.2502,  0.5918, -0.2201, -0.2832, -0.1801,
        -0.7221, -0.0857,  1.0538,  0.7574, -1.3895, -1.1931, -1.9930, -2.1967])

In [12]:
parameters_to_vector(model.parameters())

tensor([-0.2370, -0.3320, -0.2170,  0.2886, -0.0827,  0.1951,  0.1779,  0.0325,
        -0.0576, -0.0589, -0.0842,  0.0782, -0.1832, -0.0889,  0.0903, -0.0148,
        -0.0175,  0.1231,  0.1867,  0.0070, -0.0676, -0.0030, -0.3038, -0.0391,
         0.3197,  0.0278,  0.1288,  0.1186,  0.2475, -0.0284, -0.3264, -0.2566,
        -0.2124,  0.0374,  0.2054,  0.0765,  0.4812, -0.3248,  0.4979, -0.3124],
       grad_fn=<CatBackward0>)

In [13]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [14]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3
                                   )

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                #grads = grads_to_vector(model.parameters())
                optimizer.step()
                train_loss += loss.item()*data.size(0)
                
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [15]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [16]:
print(projection_list[0])

[0, tensor([[-0.3819,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.3819,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.3819,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., -0.3819,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.3819,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.3819]]), 0, tensor([[-0.6528,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.6528,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.6528,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., -0.6528,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.6528,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.6528]]), 0, 0, tensor([[1.0142, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0142, 0.0000,  ..., 0.0000, 0.0000

In [17]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(X_test, y_test), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [18]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([ 0.3126,  0.3167,  0.2975,  0.1243,  0.0321,  0.1757, -0.1198, -0.2944,
         0.1595, -0.2862, -0.1967, -0.1006,  0.1393,  0.3022,  0.1319, -0.2451,
        -0.1665, -0.2209,  0.1852, -0.1251,  0.2536,  0.1792,  0.0916, -0.2949,
        -0.2024, -0.0412,  0.3119,  0.3251, -0.3021, -0.3124, -0.2573,  0.2455,
         0.1160,  0.2282,  0.1826, -0.0095, -0.0324, -0.4335,  0.2317, -0.3184])


tensor([ 0.6251,  0.6333,  0.5951,  0.2485,  0.0643,  0.3514, -0.2397, -0.5888,
         0.3191, -0.5724, -0.3934, -0.2011,  0.2786,  0.6044,  0.2638, -0.4902,
        -0.3330, -0.4419,  0.3704, -0.2501,  0.5071,  0.3583,  0.1832, -0.5899,
        -0.4048, -0.0825,  0.6238,  0.6502, -0.6041, -0.6247, -0.5146,  0.4910,
         0.2321,  0.4564,  0.3653, -0.0190, -0.0649, -0.8671,  0.4634, -0.6369],
       grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    
    #Share and mix the local weights
    
    
    for i in range(no_users):
        weights = parameters_to_vector(dummy_models[i].parameters())
        mat_vec_sum = torch.zeros_like(weights)
        for j in G.neighbors(i):
            mat_vec_sum = torch.add(mat_vec_sum, parameters_to_vector(dummy_models[j].parameters()))
        
        new_weights = weights - mu * eta * (degree_list[i] * weights - mat_vec_sum)
        # Update real models
        vector_to_parameters(parameters=models[i].parameters(), vec=new_weights)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    #g_accuracy = sum(local_test_acc) / len(local_test_acc)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f"% (test_loss[-1]))

  0%|          | 1/2000 [00:02<1:13:20,  2.20s/it]

Training_loss 46.92816


  0%|          | 2/2000 [00:03<57:53,  1.74s/it]  

Training_loss 46.89336


  0%|          | 3/2000 [00:04<49:08,  1.48s/it]

...

Training_loss 17.46654


100%|█████████▉| 1999/2000 [38:17<00:01,  1.05s/it]

Training_loss 17.49249


100%|██████████| 2000/2000 [38:18<00:00,  1.15s/it]

Training_loss 17.48088


In [21]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([ 1.3357, -0.2490,  0.2949, -0.2663, -0.1279, -0.5051,  0.1018,  1.0945,
        -0.0629,  0.3944, -0.0255, -0.4312, -0.0193,  0.0173, -0.4775,  0.1955,
         0.0625,  0.8109,  0.0956,  0.5114, -0.4474, -0.0162,  0.6522,  0.1528,
         0.6010,  0.2775, -0.2008, -0.7141,  1.4424, -0.3200,  0.4103,  0.1399,
        -0.5005,  0.3276,  0.6776, -0.4306,  2.0459,  1.6550,  1.5424,  2.2753],
       grad_fn=<CatBackward0>)

In [22]:
for j in G.neighbors(0):
    print(j)

1
3
6
7
9


In [23]:
parameters_to_vector(models[0].parameters())

tensor([-7.7848e-01, -4.9740e-02, -4.7448e-02,  2.5144e-01,  1.7854e-01,
        -1.0460e-02,  3.1356e-01,  9.2443e-01,  4.4179e-01,  2.8600e-01,
        -1.5036e+00,  2.6853e-01, -5.0094e-01, -1.8730e-03, -3.8847e-01,
         2.9809e-01,  3.3728e-01, -5.2183e-01,  1.5949e-01, -9.5113e-01,
         2.2437e-01,  9.8373e-02,  4.3427e-01, -8.1958e-02,  4.4021e-01,
         2.9156e-01, -8.8687e-02, -1.1657e+00,  3.6460e-01,  1.6471e-01,
        -1.1556e-01, -1.3498e-01, -5.2556e-01, -5.7021e-02,  7.5317e-01,
         6.6770e-02,  1.7626e+00,  2.2856e+00,  1.7807e+00,  1.9342e+00],
       grad_fn=<CatBackward0>)

In [24]:
projection_list[0]

[0,
 tensor([[-0.3819,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.3819,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.3819,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3819,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.3819,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.3819]]),
 0,
 tensor([[-0.6528,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.6528,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.6528,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ..., -0.6528,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.6528,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.6528]]),
 0,
 0,
 tensor([[1.0142, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0142, 0.0000,  

In [25]:
projected_weights[0]

[0,
 tensor([ 0.0303, -0.1080,  0.0138, -0.0288,  0.0211,  0.0477, -0.0440, -0.0666,
          0.0580, -0.0632,  0.0655,  0.0373, -0.0216, -0.1134,  0.0888, -0.0439,
          0.0187, -0.0550, -0.0065,  0.0025,  0.0670,  0.0477,  0.0694,  0.0833,
         -0.0423, -0.0824, -0.0024,  0.0939,  0.0493,  0.1214,  0.0964, -0.0048,
          0.1188,  0.1245,  0.0578, -0.0829,  0.0345,  0.0458, -0.0358,  0.0205]),
 0,
 tensor([-0.0053,  0.0014, -0.1238, -0.1032, -0.0332,  0.1639, -0.1976, -0.0771,
         -0.0167,  0.1392,  0.0956,  0.1072,  0.0241, -0.0748,  0.1011,  0.0490,
          0.0900, -0.2000, -0.1378, -0.1166, -0.0238,  0.2148,  0.0611,  0.0522,
          0.0977,  0.0308,  0.1032, -0.1950, -0.1074,  0.0938,  0.1015, -0.1574,
         -0.0990,  0.1728, -0.2150,  0.1001,  0.3163,  0.2967,  0.3190, -0.0799]),
 0,
 0,
 tensor([-0.2044,  0.1461, -0.1306,  0.1281,  0.2286,  0.0240,  0.2531, -0.1396,
         -0.0390,  0.2523, -0.1699, -0.2225, -0.1613,  0.3110, -0.0914,  0.2371,
        

In [26]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [27]:
np.save( 'training_loss_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), test_loss)
#np.save('relative_error_dfedu' + str(eta).replace('.', '_'), total_rel_error)